# Programming for AI project 

## Phase 1: WEBSCRAPING

### 1.1. Importing libraries

In [87]:
#Importing the necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\iamha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

### 1.2 Installing chromedriver

In [88]:
# installing ChromeDriverManager
chrome_driver_path = ChromeDriverManager().install()

chrome_service = ChromeService(chrome_driver_path)

driver = webdriver.Chrome(service=chrome_service)


### 1.3. Setting up the driver

In [89]:
driver = webdriver.Chrome()
url = 'https://www.daraz.pk/'
driver.get(url)

### 1.4. Searching the data

In [90]:
# Search for Smart Phones
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Smart Phones' + Keys.RETURN)
time.sleep(2)


### 1.5 Exception Handling

In [5]:
exclude_keywords = ["case", "cover", "charger"]

# Counter for generating unique product IDs
product_id_counter = 1

def exceptions_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        # Check if any exclude keyword is present in the value
        if any(keyword.lower() in value.lower() for keyword in exclude_keywords):
            return " "
        return value
    except:
        return " "
    

def rating_exception_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        return value
    except:
        return 0.0   

### 1.6. Scraping Data from the web

In [102]:
# Counter for generating unique product IDs
product_id_counter = 1

def exceptions_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        return value
    except:
        return " "
    

def rating_exception_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        return value
    except:
        return 0.0   

In [103]:
# ----------------------------- Gathering Data ----------------------------------------------
product_id = []
product_name = []
product_price = []
product_rating = []
product_reviews = []
product_brand = []
product_specification = []
product_url = []
no_of_questions = []

def get_product_data():
    global product_id_counter
    try:
        for i in range(3, 8):
            for j in range(1, 41):
                
                # Fetch the URL of the current product
                product_element = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]/div[' + str(j) + ']/div/div/div[1]/div/a')
                product_url.append(product_element.get_attribute("href"))

                product_element.click()

                current_url = driver.current_url

                driver.execute_script("window.scrollBy(0, window.innerHeight)")
                time.sleep(3)

                current_product_id = f"PID{product_id_counter}"
                product_id.append(current_product_id)
                current_product_name = exceptions_handling(driver, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[3]/div/div/span')
                product_name.append(current_product_name)
                
                # Extract brand and handle "No Brand" case
                current_product_brand = exceptions_handling(driver, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[5]/div/a[1]')
                
                try:
                    if current_product_brand == "No Brand":
                        # If brand is "No Brand", replace it with the first word of the product name
                        current_product_brand = current_product_name.split()[0] if " " in current_product_name else current_product_name
                    product_brand.append(current_product_brand)
                except IndexError as e:
                    print(f"Extracting brand for product ID {current_product_id}: {e}")
                    product_brand.append("Brand")
                
                product_price.append(exceptions_handling(driver, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span'))
                product_rating.append(rating_exception_handling(driver, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]'))
                product_specification.append(exceptions_handling(driver, '/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div'))
                
                # no_of_questions.append(question_exception_handling(driver, '/html/body/div[4]/div/div[9]/div[1]/div[3]/div/div/div[1]/div')) 

                product_id_counter += 1

                # Go back to the previous page
                driver.back()
            next_page = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div[2]/div/ul/li[' + str(i) + ']')
            next_page.click()
            time.sleep(2)
        
        driver.execute_script("window.scrollBy(0, window.innerHeight)")

    except NoSuchElementException as e:
        print(f"Error: {e}")

get_product_data()

# Print the lengths of the lists
print("Length of product_id:", len(product_id))
print("Length of product_name:", len(product_name))
print("Length of product_price:", len(product_price))
print("Length of product_rating:", len(product_rating))
print("Length of product_brand:", len(product_brand))
print("Length of product_specification:", len(product_specification))
print("Length of product_url:", len(product_url))

# Create DataFrame only if all lists have the same length
if all(len(lst) == len(product_name) for lst in [product_id, product_name, product_price, product_rating, product_brand, product_specification, product_url]):
    df = pd.DataFrame({
        'Product ID': product_id,
        'Product Name': product_name,
        'Product Price': product_price,
        'Product Rating': product_rating,
        'Product Brand': product_brand,
        'Product Specification': product_specification,
        'Product URL': product_url
    })

else:
    print("Error: Lists have different lengths.")


Length of product_id: 200
Length of product_name: 200
Length of product_price: 200
Length of product_rating: 200
Length of product_brand: 200
Length of product_specification: 200
Length of product_url: 200


In [105]:
df.to_csv('Smart_Phones_Details.csv', index=False)
df


,Product ID,Product Name,Product Price,Product Rating,Product Brand,Product Specification,Product URL
0,PID1,Like New Phones - Used Apple iPhone X - Silver...,"Rs. 106,999",5.0,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/like-new-phones-...
1,PID2,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",4.7,Infinix,Release Date2023-05-22SIM SupportDual SIM (Nan...,https://www.daraz.pk/products/infinix-note-30-...
2,PID3,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",4.2,Realme,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/realme-narzo-50a...
3,PID4,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,"Rs. 64,999",4.7,Xiaomi,Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...,https://www.daraz.pk/products/redmi-note-12-8-...
4,PID5,Tecno SPARK 10 PRO 8GB RAM - 256GB ROM PTA APP...,"Rs. 39,999",4.5,Tecno,Starry Black/Pearl White/Magic Skin(Black and ...,https://www.daraz.pk/products/tecno-spark-10-p...
...,...,...,...,...,...,...,...
195,PID196,ZTE Blade A51 2 GB RAM 64 GB ROM 6.52 Display ...,"Rs. 16,799",5.0,ZTE,General FeaturesRelease Date2022-05-09SIM Supp...,https://www.daraz.pk/products/zte-blade-a51-2-...
196,PID197,Itel A 49 || 2GB Ram 32GB Rom || 6.6 IPS Displ...,"Rs. 17,799",5.0,itel,BuildOSAndroid 11.0 (Go edition) DimensionsN/A...,https://www.daraz.pk/products/a-49-2gb-32gb-66...
197,PID198,Xiaomi 13T Pro RAM 12 GB ROM 512 GB Front Came...,"Rs. 250,000",0.0,Xiaomi,BuildOSAndroid 13 OS UIMIUI 14 Dimensions162.2...,https://www.daraz.pk/products/13t-12-gb-512-gb...
198,PID199,Iphone 8 - 64Gb 10/10 Condition,"Rs. 34,856",0.0,Iphone,pre { white-space: pre-wrap; }• iPhone 8 - 64G...,https://www.daraz.pk/products/8-64gb-1010-i432...


### Reviews Extraction

In [173]:
driver = webdriver.Chrome()
url = 'https://www.daraz.pk/'
driver.get(url)

In [174]:
# Search for Smart Phones
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Smart Phones' + Keys.RETURN)
time.sleep(2)


In [175]:
phone_id_counter = 1

def reviews_exception_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        return value
    except:
        return "No reviews"   
    
#--------------------------------------------------------------------------    
def questions_exception_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        # Extract numeric part using regular expression
        numeric_part = re.search(r'\d+', value)
        return int(numeric_part.group()) if numeric_part else 0
    except:
        return 0
    
#---------------------------------------------------------------------------    
def total_reviews_exception_handling(element, path):
    try:
        value = element.find_element(By.XPATH, path).text
        # Extract numeric part using regular expression
        numeric_part = re.search(r'\d+', value)
        return int(numeric_part.group()) if numeric_part else 0
    except:
        return 0
    


In [178]:
# -------------------------------------------------------------- Pages ----------------------------------------------------------------------------------
phone_id = []
questions = []
Total_reviews = []
product_review = []
def get_product_data(driver):
    
    global phone_id_counter

    try:
        for i in range(3, 4):
            for j in range(1, 41):

                product_element = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[2]/div[' + str(j) + ']/div/div/div[2]/div[2]')
                product_element.click()

                driver.execute_script("window.scrollBy(0, window.innerHeight)")
                time.sleep(3)

                current_phone_id = f"PID{phone_id_counter}"
                phone_id.append(current_phone_id)

                product_review.append(reviews_exception_handling(driver, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]'))
                questions.append(questions_exception_handling(driver, '/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[2]'))
                Total_reviews.append(total_reviews_exception_handling(driver, '/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[3]'))

                phone_id_counter += 1

                # Go back to the previous page
                time.sleep(2)
                driver.back()

            next_page = driver.find_element(By.XPATH, '/html/body/div[3]/div/div[3]/div/div/div[1]/div[3]/div[2]/div/ul/li['+str(i)+']')
            next_page.click()
            driver.execute_script("window.scrollBy(0, window.innerHeight)")

    except NoSuchElementException:
        pass

    return questions, Total_reviews, product_review, phone_id

get_product_data(driver)

print("Length of phone_id:", len(phone_id))
print("Length of questions:", len(questions))
print("Length of Total_reviews:", len(Total_reviews))
print("Length of product_review:", len(product_review))

# Check if lists have the same length
if all(len(lst) == len(questions) for lst in [phone_id, questions, Total_reviews, product_review]):
    df = pd.DataFrame({

        'Product ID': phone_id,
        'Reviews': product_review,
        'Questions': questions,
        'Total reviews': Total_reviews
    })
    
else:
    print("Error: Lists have different lengths.")

Length of phone_id: 203
Length of questions: 203
Length of Total_reviews: 203
Length of product_review: 203


In [179]:
df.to_csv('Smart_Phones_Reviews.csv', index=False)
df

,Product ID,Reviews,Questions,Total reviews
0,PID1,Gohar I.\n4 months ago\n...\nthis is the initi...,0,401
1,PID2,Snake E.\n3 weeks ago\n...\nI got what was sho...,217,152
2,PID3,Habib R.\n2 weeks ago\n...\n#11.11 PTA approve...,140,92
3,PID4,Usman M.\n2 weeks ago\n...\nReceived as promis...,0,1
4,PID5,Hussain\n4 months ago\n...\nSo far it's good. ...,0,198
...,...,...,...,...
198,PID116,Ali Z.\n5 months ago\n...\n👍\nSeller Response\...,91,2
199,PID117,N***.\n6 months ago\n...\nماشاءاللہ بہت زبردست...,16,28
200,PID118,Babar H.\n5 months ago\n...\nI had a great exp...,0,3
201,PID119,No reviews,0,0


## Data cleaning and Mergning the data

In [4]:
import pandas as pd

file1_path = 'Smart_Phones_Details.csv'
file2_path = 'Smart_Phones_Reviews.csv'

df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Identify and remove non-unique columns
non_unique_columns = df1.columns[df1.columns.isin(df2.columns)]
df2 = df2.loc[:, ~df2.columns.isin(non_unique_columns)]

products_details = pd.concat([df1, df2], axis=1)

# Remove rows with no values
products_details = products_details.dropna()

products_details.to_csv('products_details.csv', index=False)

print(products_details.head())


  Product ID                                       Product Name Product Price  \
0       PID1  Like New Phones - Used Apple iPhone X - Silver...   Rs. 106,999   
1       PID2  Infinix Note 30 Pro 8-256 GB PTA Approved With...    Rs. 64,499   
2       PID3  Realme Narzo 50A Prime (4GB-128GB) PTA Approve...    Rs. 32,999   
3       PID4  Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...    Rs. 64,999   
4       PID5  Tecno SPARK 10 PRO 8GB RAM - 256GB ROM PTA APP...    Rs. 39,999   

   Product Rating Product Brand  \
0             5.0         Apple   
1             4.7       Infinix   
2             4.2        Realme   
3             4.7        Xiaomi   
4             4.5         Tecno   

                               Product Specification  \
0  Model: Apple\nModel: iPhone X\nBattery Health ...   
1  Release Date2023-05-22SIM SupportDual SIM (Nan...   
2  ColourXanh Tia Chop = Lightning Blueđen = Blac...   
3  Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...   
4  Starry Black/Pearl 

In [11]:
df = pd.read_csv('products_details.csv')
df['Product Price'] = df['Product Price'].str.replace('Rs.', '')
df['Product Price'] = df['Product Price'].str.replace(',', '').astype(int)
# save the cleaned data
df.to_csv('products_details.csv', index=False) 
df.head()


,Product ID,Product Name,Product Price,Product Rating,Product Brand,Product Specification,Product URL,Reviews,Questions,Total reviews
0,PID1,Like New Phones - Used Apple iPhone X - Silver...,106999,5.0,Apple,Model: Apple\nModel: iPhone X\nBattery Health ...,https://www.daraz.pk/products/like-new-phones-...,Gohar I.\n4 months ago\n...\nthis is the initi...,0,401
1,PID2,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,4.7,Infinix,Release Date2023-05-22SIM SupportDual SIM (Nan...,https://www.daraz.pk/products/infinix-note-30-...,Snake E.\n3 weeks ago\n...\nI got what was sho...,217,152
2,PID3,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,4.2,Realme,ColourXanh Tia Chop = Lightning Blueđen = Blac...,https://www.daraz.pk/products/realme-narzo-50a...,Habib R.\n2 weeks ago\n...\n#11.11 PTA approve...,140,92
3,PID4,Redmi Note 12 8 GB RAM + 128GB ROM PTA Approve...,64999,4.7,Xiaomi,Redmi Note 12 8GB RAM + 128GB ROM PTA Approved...,https://www.daraz.pk/products/redmi-note-12-8-...,Usman M.\n2 weeks ago\n...\nReceived as promis...,0,1
4,PID5,Tecno SPARK 10 PRO 8GB RAM - 256GB ROM PTA APP...,39999,4.5,Tecno,Starry Black/Pearl White/Magic Skin(Black and ...,https://www.daraz.pk/products/tecno-spark-10-p...,Hussain\n4 months ago\n...\nSo far it's good. ...,0,198


In [10]:
# find total number of products
total_products = len(df)
print("Total number of products:", total_products)
#  find the average price of all the products
avg_price = df['Product Price'].mean().astype(int) 
print("Average price of all the products:", avg_price)
# find the average rating of all the products
avg_rating = df['Product Rating'].mean().astype(float) 
print("Average rating of all the products:", avg_rating)
# find the average number of reviews of all the products
avg_reviews = df['Total reviews'].mean().astype(int) 
print("Average number of reviews of all the products:", avg_reviews)
# find the total number of questions of all the products
total_questions = df['Questions'].sum()
print("Total number of questions of all the products:", total_questions)

Total number of products: 190
Average price of all the products: 56315
Average rating of all the products: 3.040526315789474
Average number of reviews of all the products: 20
Total number of questions of all the products: 4146


## Creating a database

In [6]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('Smart_phones_Database.db')

df_products = pd.read_csv('Smart_Phones_Details.csv')
df_reviews = pd.read_csv('Smart_Phones_Reviews.csv')

df_products.to_sql('Products', conn, if_exists='replace', index=False)
df_reviews.to_sql('Reviews', conn, if_exists='replace', index=False)

cursor = conn.cursor()
cursor.execute("SELECT * FROM Products")
products_data = cursor.fetchall()

print("Products table:")
for row in products_data:
    print(row)

cursor.execute("SELECT * FROM Reviews")
reviews_data = cursor.fetchall()

print("\nReviews table:")
for row in reviews_data:
    print(row)

conn.close()

Products table:
('PID1', 'Like New Phones - Used Apple iPhone X - Silver 256 GB - PTA Approved', 'Rs. 106,999', 5.0, 'Apple', "Model: Apple\nModel: iPhone X\nBattery Health - 80+\nPTA Approved\nScreen: 5.8-in. Super Retina HD\nCamera: 12MP Zoom 10x\nVideo camera: 4K Zoom 6x\nProcessor: A11 Bionic chip\nMemory: 3GB\nBattery: Lithium-ion 2716 mAh\nGenuine OEM components\nCertified 6 Months Daraz Warranty.\nDaraz Warranty Center Location: Karachi & Lahore.\nWarranty will be claimed on order number/invoice.\nThe phone will be dispatched in Daraz's Like New packaging.\nPhone will be slightly used, ensuring its authenticity with a genuine A-grade kit.\nThe original box, charging cable and charging dock will not be included.\nAll Daraz Like New Phones are thoroughly checked by experts to ensure your product is original, has all genuine parts and is in A+ condition.\n60+ Quality Check Points – International Standard Grading software, quality check report will also be sent.\nFully functional Fa